In [4]:
import openai
import json
import pandas as pd

openai.api_key = 'hello'

def extract_financial_data(text):
    prompt = get_prompt_financial() + text
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user","content": prompt}]
    )
    content = response.choices[0]['message']['content']

    try:
        data = json.loads(content)
        return pd.DataFrame(data.items(), columns=["Measure", "Value"])

    except (json.JSONDecodeError, IndexError):
        pass

    return pd.DataFrame({
        "Measure": ["Company Name", "Stock Symbol", "Revenue", "Net Income", "EPS"],
        "Value": ["", "", "", "", ""]
    })

In [5]:
import streamlit as st
import pandas as pd
import openai_helper

col1, col2 = st.columns([3,2])

financial_data_df = pd.DataFrame({
        "Measure": ["Company Name", "Stock Symbol", "Revenue", "Net Income", "EPS"],
        "Value": ["", "", "", "", ""]
    })

with col1:
    st.title("Data Extraction Tool")
    news_article = st.text_area("Paste your financial news article here", height=300)
    if st.button("Extract"):
        financial_data_df = openai_helper.extract_financial_data(news_article)

with col2:
    st.markdown("<br/>" * 5, unsafe_allow_html=True)  # Creates 5 lines of vertical space
    st.dataframe(
        financial_data_df,
        column_config={
            "Measure": st.column_config.Column(width=150),
            "Value": st.column_config.Column(width=150)
        },
        hide_index=True
    )

2024-02-04 02:51:57.524 
  command:

    streamlit run /opt/homebrew/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [6]:
def get_prompt_financial():
    return '''Please retrieve company name, revenue, net income and earnings per share (a.k.a. EPS)
    from the following news article. If you can't find the information from this article 
    then return "". Do not make things up.    
    Then retrieve a stock symbol corresponding to that company. For this you can use
    your general knowledge (it doesn't have to be from this article). Always return your
    response as a valid JSON string. The format of that string should be this, 
    {
        "Company Name": "Walmart",
        "Stock Symbol": "WMT",
        "Revenue": "12.34 million",
        "Net Income": "34.78 million",
        "EPS": "2.1 $"
    }
    News Article:
    ============

    '''

# Press the green button in the gutter to run the script.
if __name__ == '__main__':
    text = '''
    Tesla's Earning news in text format: Tesla's earning this quarter blew all the estimates. They reported 4.5 billion $ profit against a revenue of 30 billion $. Their earnings per share was 2.3 $
    '''
    df = extract_financial_data(text)

    print(df.to_string())

2024-02-04 02:52:01,525 : DEBUG : util.py : log_debug() : 60 : message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
2024-02-04 02:52:01,526 : DEBUG : util.py : log_debug() : 60 : api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "Please retrieve company name, revenue, net income and earnings per share (a.k.a. EPS)\\n    from the following news article. If you can\'t find the information from this article \\n    then return \\"\\". Do not make things up.    \\n    Then retrieve a stock symbol corresponding to that company. For this you can use\\n    your general knowledge (it doesn\'t have to be from this article). Always return your\\n    response as a valid JSON string. The format of that string should be this, \\n    {\\n        \\"Company Name\\": \\"Walmart\\",\\n        \\"Stock Symbol\\": \\"WMT\\",\\n        \\"Revenue\\": \\"12.34 million\\",\\n        \\"Net Income\\": \\"34.78 million\\",\\n      

AuthenticationError: Incorrect API key provided: hello. You can find your API key at https://platform.openai.com/account/api-keys.